### http протокол
2 стороны участвуют в общении:
   - клиент (мы)
   - сервер
   
URL - "единообразный локатор ресурсов"

In [1]:
import requests

In [2]:
res = requests.get('https://docs.python.org/3.5/')
print(res.status_code)
print(res.headers['Content-Type'])

200
text/html


In [5]:
#print(res.content)
#print(res.text)

In [7]:
res = requests.get('https://yandex.ru/search', params={'text' : 'Stepic'})
print(res.status_code)
print(res.headers['Content-Type'])
print(res.url)
#print(res.text)

200
text/html; charset=utf-8
https://yandex.ru/search/?text=Stepic&lr=35&redircnt=1585161654.1


### Задача

- Рассмотрим два HTML-документа A и B.


- Из A можно перейти в B за один переход, если в A есть ссылка на B, т. е. внутри A есть тег < a href="B">, возможно с дополнительными параметрами внутри тега.
- Из A можно перейти в B за два перехода если существует такой документ C, что из A в C можно перейти за один переход и из C в B можно перейти за один переход.

Вашей программе на вход подаются две строки, содержащие url двух документов A и B.
Выведите Yes, если из A в B можно перейти за два перехода, иначе выведите No.

### Первая попытка

In [30]:
import re 

A, B = input(), input()

res_A = requests.get(A)
res_B = requests.get(B)

def find_url_lst(res):
    s = r'<a href=".*">'
    links = re.findall(s, res.text)
    return links

def find_url(url_input):
    s = r'".*"'
    url = re.findall(s, url_input)
    return url
    
flag = False
if res_A.status_code == 200 and res_B.status_code == 200:
    links = find_url_lst(res_A)
    
    for l in links:
        u = find_url(l)
        res_new = requests.get(u[0][1:-1])
        if res_new.status_code == 200:
                s = r'<a href="{}">'.format(B)
                links = re.findall(s, res_new.text)
                if len(links) > 0:
                    flag = True
                    break
if flag:
    print('Yes')
else:
    print('No')
        

https://stepic.org/media/attachments/lesson/24472/sample0.html
https://stepic.org/media/attachments/lesson/24472/sample2.html
Yes


### Вторая попытка

In [33]:
import re 
import requests

A, B = input(), input()

res_A = requests.get(A)
res_B = requests.get(B)

def find_url_lst(res):
    s = r'<a href=".*">'
    links = re.findall(s, res.text)
    upd_links = [l[9:-2] for l in links]
    return upd_links

def get_result():
    if res_A.status_code == 200 and res_B.status_code == 200:
        url_in_A = find_url_lst(res_A)
        for l in url_in_A:
            res_C = requests.get(l)
            if res_C.status_code == 200:
                s = r'<a href="{}">'.format(B)
                if re.search(s, res_C.text):
                    return True
    return False

if get_result():
    print('Yes')
else:
    print('No')

https://stepic.org/media/attachments/lesson/24472/sample0.html
https://stepic.org/media/attachments/lesson/24472/sample2.html
Yes
Yes


### Попытка №3 - лучшее решение

In [ ]:
import re
import requests

start_url = input()
end_url = input()

found = False

link_pattern = re.compile(r'<a[^>]*?href="(.*?)"[^>]*?>')

resp = requests.get(start_url).text
for url in link_pattern.findall(resp):
    cur_resp = requests.get(url).text
    if end_url in link_pattern.findall(cur_resp):
        found = True
        break

print("Yes" if found else "No")


### ИЛИ

import requests, re

urls = [input() for cmd in range(2)]
p = 'No'

for i in re.findall(r'<a.*href="(.*)">', requests.get(urls[0]).text):
    if urls[1] in re.findall(r'<a.*href="(.*)">', requests.get(i).text):
        p = 'Yes'
        break

print(p)